In [1]:
import sys
sys.path.insert(0, '../')

import time
import torch
from torch.distributions import Categorical, kl
from d2l.torch import Animator

from aco import ACO
from utils import gen_pyg_data, load_test_dataset

from tsp.net import Net as Singlehead_net
from tsp.aco import ACO as Singlehead_ACO
from net import Net as Multihead_net
from aco import ACO as Multihead_ACO

torch.manual_seed(1234)

n_heads = 4
EPS = 1e-10
device = 'cuda:0'

In [2]:
@torch.no_grad()
def infer_instance(model, pyg_data, distances, n_ants, t_aco_diff, multi):
    model.eval()
    if not multi:
        heu_vec = model(pyg_data)
        heu_mat = model.reshape(pyg_data, heu_vec) + EPS
        aco = Singlehead_ACO(
            n_ants=n_ants,
            heuristic=heu_mat,
            distances=distances,
            device=device
        )
    else:
        heu_vec_list = model(pyg_data)
        heu_mat_list = [model.reshape(pyg_data, heu_vec_list[i]) + EPS for i in range(n_heads)]
        aco = Multihead_ACO(
            n_ants=n_ants,
            heuristic=heu_mat_list,
            distances=distances,
            device=device
        )
    results = torch.zeros(size=(len(t_aco_diff),), device=device)
    for i, t in enumerate(t_aco_diff):
        best_cost = aco.run(t)
        results[i] = best_cost
    return results
        
    
@torch.no_grad()
def test(dataset, model, n_ants, t_aco, multi):
    _t_aco = [0] + t_aco
    t_aco_diff = [_t_aco[i+1]-_t_aco[i] for i in range(len(_t_aco)-1)]
    sum_results = torch.zeros(size=(len(t_aco_diff),), device=device)
    start = time.time()
    for pyg_data, distances in dataset:
        results = infer_instance(model, pyg_data, distances, n_ants, t_aco_diff, multi)
        sum_results += results
    end = time.time()
    
    return sum_results / len(dataset), end-start

### Test on TSP20

multihead

In [3]:
n_ants = 20
n_node = 20
k_sparse = 10
t_aco = [1, 50, 100, 150, 200, 250]
test_list = load_test_dataset(n_node, k_sparse, device)
net_tsp = Multihead_net(n_heads).to(device)
net_tsp.load_state_dict(torch.load(f'../pretrained/tsp-spatial/tsp{n_node}.pt', map_location=device))
avg_aco_best, duration = test(test_list, net_tsp, n_ants, t_aco, True)
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))    

total duration:  801.2649049758911
T=1, average cost is 3.9010469913482666.
T=50, average cost is 3.808290719985962.
T=100, average cost is 3.804656982421875.
T=150, average cost is 3.804598093032837.
T=200, average cost is 3.8034627437591553.
T=250, average cost is 3.8034627437591553.


singlehead

In [7]:
n_ants = 20
n_node = 20
k_sparse = 10
t_aco = [1, 50, 100, 150, 200, 250]
test_list = load_test_dataset(n_node, k_sparse, device)
net_tsp = Singlehead_net().to(device)
net_tsp.load_state_dict(torch.load(f'../pretrained/tsp/tsp{n_node}.pt', map_location=device))
avg_aco_best, duration = test(test_list, net_tsp, n_ants, t_aco, multi=False)
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))    

total duration:  341.8486297130585
T=1, average cost is 3.9362034797668457.
T=50, average cost is 3.8097853660583496.
T=100, average cost is 3.807478904724121.
T=150, average cost is 3.8070151805877686.
T=200, average cost is 3.8065853118896484.
T=250, average cost is 3.8063125610351562.


### Test on TSP100

multihead

In [12]:
n_ants = 100
n_node = 100
k_sparse = 20
t_aco = [1, 10, 20, 30, 40, 50, 100]
test_list = load_test_dataset(n_node, k_sparse, device)
net_tsp = Multihead_net(n_heads).to(device)
net_tsp.load_state_dict(torch.load(f'../pretrained/tsp-spatial/tsp{n_node}.pt', map_location=device))
avg_aco_best, duration = test(test_list, net_tsp, n_ants, t_aco, multi=True)
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))    

total duration:  1599.5140709877014
T=1, average cost is 8.728137016296387.
T=10, average cost is 8.289116859436035.
T=20, average cost is 8.217846870422363.
T=30, average cost is 8.193182945251465.
T=40, average cost is 8.177783012390137.
T=50, average cost is 8.166511535644531.
T=100, average cost is 8.127159118652344.


singlehead

In [13]:
n_ants = 100
n_node = 100
k_sparse = 20
t_aco = [1, 10, 20, 30, 40, 50, 100]
test_list = load_test_dataset(n_node, k_sparse, device)
net_tsp = Singlehead_net().to(device)
net_tsp.load_state_dict(torch.load(f'../pretrained/tsp/tsp{n_node}.pt', map_location=device))
avg_aco_best, duration = test(test_list, net_tsp, n_ants, t_aco, multi=False)
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))    

total duration:  682.2962992191315
T=1, average cost is 8.816386222839355.
T=10, average cost is 8.331635475158691.
T=20, average cost is 8.264138221740723.
T=30, average cost is 8.230011940002441.
T=40, average cost is 8.210904121398926.
T=50, average cost is 8.19095230102539.
T=100, average cost is 8.155533790588379.
